# Задание 1.2
Напишите запрос, чтобы найти всех активных сотрудников и наименования магазинов, в которых они работают.

(Представлены два способа решения)

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").getOrCreate()


API

In [6]:
from pyspark.sql.functions import concat_ws

df_staffs = spark.read.csv("CVS\\staffs.csv", inferSchema=True, header=True, sep=",")

df_stores = spark.read.csv("CVS\\stores.csv", inferSchema=True, header=True, sep=",")

df_res = df_staffs.join(df_stores, 
                        df_stores["store_id"] == df_staffs["store_id"]
                        ).where(df_staffs["active"] == 1)

df_res = df_res.select(concat_ws(" ",df_staffs["first_name"], 
                                 df_staffs["last_name"]).alias("staff"),
                                 "store_name")

df_res.orderBy(['staff'], ascending = [True]).show()

+------------------+----------------+
|             staff|      store_name|
+------------------+----------------+
|Bernardine Houston|   Rowlett Bikes|
|   Fabiola Jackson|Santa Cruz Bikes|
|     Genna Serrano|Santa Cruz Bikes|
|    Jannette David|   Baldwin Bikes|
|       Kali Vargas|   Rowlett Bikes|
|     Layla Terrell|   Rowlett Bikes|
|   Marcelene Boyer|   Baldwin Bikes|
|   Mireya Copeland|Santa Cruz Bikes|
|     Venita Daniel|   Baldwin Bikes|
|    Virgie Wiggins|Santa Cruz Bikes|
+------------------+----------------+



spark.sql

In [7]:
df_staffs.createOrReplaceTempView("staffs")
df_stores.createOrReplaceTempView("stores")

spark.sql("""
          SELECT CONCAT_WS(' ',staffs.first_name, staffs.last_name) as staff, stores.store_name
          FROM staffs
          JOIN stores ON stores.store_id = staffs.store_id
          WHERE staffs.active = 1
          order by staff
          """).show()

+------------------+----------------+
|             staff|      store_name|
+------------------+----------------+
|Bernardine Houston|   Rowlett Bikes|
|   Fabiola Jackson|Santa Cruz Bikes|
|     Genna Serrano|Santa Cruz Bikes|
|    Jannette David|   Baldwin Bikes|
|       Kali Vargas|   Rowlett Bikes|
|     Layla Terrell|   Rowlett Bikes|
|   Marcelene Boyer|   Baldwin Bikes|
|   Mireya Copeland|Santa Cruz Bikes|
|     Venita Daniel|   Baldwin Bikes|
|    Virgie Wiggins|Santa Cruz Bikes|
+------------------+----------------+

